# Rozwiązania zadań z analizy ramek semantycznych i skryptów

W tym notebooku przedstawimy rozwiązania zadań dotyczących analizy tekstu z wykorzystaniem ramek semantycznych.

In [ ]:
import nltk
from nltk.corpus import framenet as fn
import spacy
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

# Pobieranie potrzebnych zasobów
nltk.download('framenet_v17')

# Wczytanie modeli spaCy
nlp_sm = spacy.load("en_core_web_sm")
nlp_lg = spacy.load("en_core_web_lg")

## 1. Identyfikacja ról semantycznych

In [ ]:
def identify_semantic_roles(sentence):
    doc = nlp_sm(sentence)
    roles = defaultdict(list)
    
    for token in doc:
        if token.dep_ in ['nsubj', 'dobj', 'iobj']:
            # Mapowanie zależności składniowych na role semantyczne
            if token.dep_ == 'nsubj':
                role = 'Agent'
            elif token.dep_ == 'dobj':
                role = 'Patient'
            else:
                role = 'Recipient'
                
            roles[token.head.text].append((token.text, role))
            
    return dict(roles)

# Przykład użycia
sentence = "The chef cooked a delicious meal for his guests."
semantic_roles = identify_semantic_roles(sentence)
print("Role semantyczne:")
for verb, args in semantic_roles.items():
    print(f"\nCzasownik: {verb}")
    for arg, role in args:
        print(f"- {arg}: {role}")

## 2. Wizualizacja ramek semantycznych

In [ ]:
def visualize_semantic_frames(frames):
    G = nx.DiGraph()
    
    for verb, frame in frames:
        # Dodanie węzła czasownika
        G.add_node(verb, node_type='verb')
        # Dodanie węzła ramki
        G.add_node(frame, node_type='frame')
        # Dodanie krawędzi
        G.add_edge(verb, frame)
        
        # Dodanie elementów ramki
        frame_info = fn.frame(frame)
        for fe in frame_info.FE.values():
            if fe.coreType == 'Core':
                G.add_node(fe.name, node_type='element')
                G.add_edge(frame, fe.name)
    
    # Rysowanie grafu
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G)
    
    # Rysowanie węzłów różnych typów
    nx.draw_networkx_nodes(G, pos, 
                          node_color='lightblue',
                          node_size=2000,
                          nodelist=[n for n,d in G.nodes(data=True) if d['node_type']=='verb'])
    nx.draw_networkx_nodes(G, pos,
                          node_color='lightgreen',
                          node_size=1500,
                          nodelist=[n for n,d in G.nodes(data=True) if d['node_type']=='frame'])
    nx.draw_networkx_nodes(G, pos,
                          node_color='lightpink',
                          node_size=1000,
                          nodelist=[n for n,d in G.nodes(data=True) if d['node_type']=='element'])
    
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos)
    
    plt.title("Wizualizacja ramek semantycznych")
    plt.axis('off')
    plt.show()

# Przykład użycia
identified_frames = identify_semantic_frames(sentence)
visualize_semantic_frames(identified_frames)

## 3. Analiza dłuższego tekstu

In [ ]:
longer_text = """
The skilled chef carefully prepared a magnificent feast in the restaurant's kitchen. 
He chopped fresh vegetables and seasoned the meat with exotic spices. 
The sous chef assisted him while the waiters arranged the dining room. 
Finally, they served the delicious dishes to the eager guests.
"""

def analyze_text(text):
    sentences = [sent.text.strip() for sent in nlp_sm(text).sents]
    results = []
    
    for sentence in sentences:
        frames = identify_semantic_frames(sentence)
        roles = identify_semantic_roles(sentence)
        results.append({
            'sentence': sentence,
            'frames': frames,
            'roles': roles
        })
    
    return results

analysis_results = analyze_text(longer_text)
for result in analysis_results:
    print(f"\nZdanie: {result['sentence']}")
    print("Ramki:")
    for verb, frame in result['frames']:
        print(f"- {verb}: {frame}")
    print("Role:")
    for verb, roles in result['roles'].items():
        print(f"- {verb}: {roles}")

## 4. Analiza różnych typów zdań

In [ ]:
different_sentences = {
    'oznajmujące': "The chef prepares the meal.",
    'pytające': "What is the chef preparing?",
    'rozkazujące': "Prepare the meal quickly!",
    'wykrzyknikowe': "How wonderfully the chef cooks!"
}

for sent_type, sent in different_sentences.items():
    print(f"\nTyp zdania: {sent_type}")
    print(f"Zdanie: {sent}")
    
    frames = identify_semantic_frames(sent)
    roles = identify_semantic_roles(sent)
    
    print("Ramki:")
    for verb, frame in frames:
        print(f"- {verb}: {frame}")
    print("Role:")
    for verb, role_list in roles.items():
        print(f"- {verb}: {role_list}")

## 5. Porównanie modeli spaCy

In [ ]:
def compare_spacy_models(sentence):
    print(f"Zdanie: {sentence}\n")
    
    # Analiza z użyciem modelu small
    doc_sm = nlp_sm(sentence)
    print("Model en_core_web_sm:")
    for token in doc_sm:
        print(f"Token: {token.text}, POS: {token.pos_}, DEP: {token.dep_}")
    
    print("\nModel en_core_web_lg:")
    # Analiza z użyciem modelu large
    doc_lg = nlp_lg(sentence)
    for token in doc_lg:
        print(f"Token: {token.text}, POS: {token.pos_}, DEP: {token.dep_}")

test_sentence = "The experienced chef masterfully prepared the complex dish."
compare_spacy_models(test_sentence)